In [1]:
# from functions import *
# for lambda_val in [0.0, 1.0, 2.0, 3.0, 3.25, 3.5, 3.75, 4.0]:
#     result_dir = "../../exp_results_test_20240328_30ms_" + str(lambda_val) + "lambda"
    
#     summary_path = result_dir + "/result_summary.csv"
#     output_path = result_dir + "/correctness_per_dataset.csv"

#     print("\nlambda : ", lambda_val)
#     check_correntness(summary_path, output_path)

In [2]:
# compare with the simulator
from functions import *
result_dir = "../../exp_results_test_20240328_30ms_final_5"
    
summary_path = result_dir + "/result_summary.csv"
output_path = result_dir + "/correctness_per_dataset.csv"

check_correntness(summary_path, output_path)

    numTasks  utilization   TP  TN  FP  FN  accuracy
0        3.0          0.4  100   0   0   0      1.00
1        3.0          0.6  100   0   0   0      1.00
2        3.0          0.8   98   0   0   2      0.98
3        6.0          0.4  100   0   0   0      1.00
4        6.0          0.6   97   0   0   3      0.97
5        6.0          0.8   96   3   0   1      0.99
6        9.0          0.4  100   0   0   0      1.00
7        9.0          0.6   94   4   0   2      0.98
8        9.0          0.8   79  13   0   8      0.92
9       12.0          0.4   87  13   0   0      1.00
10      12.0          0.6   52  46   0   2      0.98
11      12.0          0.8   34  62   0   4      0.96
num_TP :  1037
total accuracy :  0.9816666666666668


In [4]:
# compare with real linux
from functions import *
import pandas as pd
proposed_result_path = "../../exp_results_test_20240328_30ms_final_7_real/result_summary.csv"
real_result_path = "../../scheduler_comparision/app/exp_results/summary.csv"
    
proposed_df = pd.read_csv(proposed_result_path, sep=",")
real_df = pd.read_csv(real_result_path, sep=",")

# filter proposed_df by the same configurations in real_df
proposed_df = proposed_df[proposed_df['tasksetIndex'] < 30]
proposed_df = proposed_df[['numCores',  'numTasks',  'utilization', 'tasksetIndex', 'proposed_schedulability']]

# merge proposed_df and real_df with ['numCores',  'numTasks',  'utilization', 'tasksetIndex']
merged_df = pd.merge(proposed_df, real_df, on=['numCores',  'numTasks',  'utilization', 'tasksetIndex'])

merged_df = merged_df.drop(columns=['numCores'])

results = []
configurations = proposed_df[['numTasks', 'utilization']].drop_duplicates()
for i, (num_tasks, utilization) in configurations.iterrows():
        subset = merged_df[(merged_df['numTasks'] == num_tasks) & (merged_df['utilization'] == utilization)]
        
        # Calculate confusion matrix for the subset
        confusion = confusion_matrix(subset['realLinux_schedulability'], subset['proposed_schedulability'])
        
        # Calculate True Positives (TP), True Negatives (TN), False Positives (FP), and False Negatives (FN)
        if (confusion.shape == (2, 2)):
            TP = confusion[1, 1]
            TN = confusion[0, 0]
            FP = confusion[0, 1]
            FN = confusion[1, 0]
        else:
            assert confusion.shape == (1, 1), "need to check the confusion matrix : confusion.shape" + str(confusion.shape)
            if subset['realLinux_schedulability'].iloc[0] == False:
                TP = 0
                TN = confusion[0, 0]
                FP = 0
                FN = 0
            else:
                TP = confusion[0, 0]
                TN = 0
                FP = 0
                FN = 0
                
        accuracy = (TP + TN) / (TP + TN + FP + FN)
        # precision = TP / (TP + FP) if (TP + FP) != 0 else 0
        # recall = TP / (TP + FN) if (TP + FN) != 0 else 0

        # Append the results to the list
        results.append([num_tasks, utilization, TP, TN, FP, FN, accuracy])

results_df = pd.DataFrame(results, columns=['numTasks', 'utilization', 'TP', 'TN', 'FP', 'FN', 'accuracy'])
results_df = results_df.sort_values(['numTasks', 'utilization'])
print(results_df)
total_accuracy = results_df['accuracy'].mean()
print("total accuracy : ", total_accuracy)



    numTasks  utilization  TP  TN  FP  FN  accuracy
0        3.0          0.4  30   0   0   0  1.000000
1        3.0          0.6  30   0   0   0  1.000000
2        3.0          0.8  30   0   0   0  1.000000
3        6.0          0.4  30   0   0   0  1.000000
4        6.0          0.6  28   0   0   2  0.933333
5        6.0          0.8  28   0   0   2  0.933333
6        9.0          0.4  30   0   0   0  1.000000
7        9.0          0.6  28   0   0   2  0.933333
8        9.0          0.8  20   0   0  10  0.666667
9       12.0          0.4  26   0   0   4  0.866667
10      12.0          0.6  15   0   0  15  0.500000
11      12.0          0.8   6   0   0  24  0.200000
total accuracy :  0.8361111111111111


In [4]:
print(merged_df[(merged_df['proposed_schedulability'] == True) & (merged_df['realLinux_schedulability'] == False)])

Empty DataFrame
Columns: [numTasks, utilization, tasksetIndex, proposed_schedulability, realLinux_schedulability]
Index: []


In [5]:
# import pandas as pd

# result_dir = "../../exp_results_test_20240328_30ms_3.25lambda"

# input_path = result_dir + "/result_summary.csv"

# df = pd.read_csv(input_path, sep=",")

# results = []
# combinations = df[['numTasks', 'utilization']].drop_duplicates()

# for i, (num_tasks, utilization) in combinations.iterrows():
#     subset = df[(df['numTasks'] == num_tasks) & (df['utilization'] == utilization)]
    
#     # count the number of schedulable tasks by proposed algorithm
#     num_sched_CFS = subset[subset['proposed_schedulability'] == True].shape[0]
#     num_sched_FIFO = subset[subset['FIFO_schedulability'] == True].shape[0]
#     num_sched_RR = subset[subset['RR_schedulability'] == True].shape[0]
#     num_sched_RM = subset[subset['RM_schedulability'] == True].shape[0]
    
#     results.append([num_tasks, utilization, num_sched_CFS, num_sched_FIFO, num_sched_RR, num_sched_RM])

# results_df = pd.DataFrame(results, columns=['numTasks', 'utilization', 'CFS', 'FIFO', 'RR', 'RM'])
# results_df = results_df.sort_values(['numTasks', 'utilization'])
# print(results_df)
    